In [74]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 5, 74, Finished, Available)

In [75]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("EscolaridadeVitima", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 5, 75, Finished, Available)

In [76]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Escolaridade Vitima.csv')

StatementMeta(tcccibele, 5, 76, Finished, Available)

In [77]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 5, 77, Finished, Available)

In [78]:
#Remove "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'EscolaridadeVitima':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 5, 78, Finished, Available)

In [79]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/EscolaridadeVitima/')

StatementMeta(tcccibele, 5, 79, Finished, Available)

In [80]:
df = spark.read.parquet('/history_zone/2019/EscolaridadeVitima/')
df.toPandas()

StatementMeta(tcccibele, 5, 80, Finished, Available)

,EscolaridadeVitima,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,Analfabeto(a),237,202,201,168,143,141,121,169,161,145,151,82
1,Ensino Fundamental Incompleto,1999,1691,1830,1653,1629,1431,1513,1756,1634,1600,1416,758
2,Ensino Fundamental completo,611,465,497,415,428,396,417,447,440,412,377,197
3,Ensino Médio Incompleto,823,640,761,687,684,523,615,723,608,696,610,286
4,Ensino Médio completo,2421,1903,2062,1862,1843,1653,1752,2063,1928,1874,1784,908
5,Nível Técnico,0,0,0,0,0,0,0,0,0,0,0,0
6,Ensino Superior Incompleto,523,447,477,460,438,410,400,462,491,499,496,241
7,Ensino Superior completo,786,685,758,600,634,562,541,660,661,619,598,346
8,Pós Graduação,0,0,0,0,0,0,0,0,0,0,0,0
9,Especialização,85,79,90,72,65,49,63,77,77,75,60,38
